## Preparing directory

In [ ]:
import os,shutil,numpy as np
sl=os.listdir("/content/drive/My Drive/DM Course Project/Plot images/seizure") 
nsl=os.listdir("/content/drive/My Drive/DM Course Project/Plot images/non-seizure") 
print("seizure: "+str(len(sl)))
print("non-seizure: "+str(len(nsl)))

seizure: 5618
non-seizure: 6473


In [ ]:
print(sl)

['chb15_62_48.png', 'chb15_62_49.png', 'chb15_62_50.png', 'chb15_62_51.png', 'chb15_62_52.png', 'chb15_62_53.png', 'chb15_62_54.png', 'chb15_62_55.png', 'chb15_62_56.png', 'chb15_62_57.png', 'chb15_62_58.png', 'chb15_62_59.png', 'chb15_62_60.png', 'chb15_62_61.png', 'chb15_62_62.png', 'chb15_62_63.png', 'chb15_62_64.png', 'chb15_62_65.png', 'chb15_62_66.png', 'chb15_62_67.png', 'chb15_62_68.png', 'chb15_62_69.png', 'chb15_62_70.png', 'chb15_62_71.png', 'chb15_62_72.png', 'chb15_62_73.png', 'chb15_62_74.png', 'chb15_62_75.png', 'chb15_62_76.png', 'chb15_62_77.png', 'chb15_62_78.png', 'chb15_62_79.png', 'chb15_62_80.png', 'chb15_62_81.png', 'chb15_62_82.png', 'chb15_62_83.png', 'chb15_62_84.png', 'chb15_62_85.png', 'chb15_62_86.png', 'chb15_62_87.png', 'chb15_62_88.png', 'chb15_62_89.png', 'chb15_62_90.png', 'chb15_62_91.png', 'chb15_62_92.png', 'chb15_62_93.png', 'chb15_62_94.png', 'chb15_62_95.png', 'chb15_62_96.png', 'chb15_62_97.png', 'chb15_62_98.png', 'chb15_62_99.png', 'chb15_62_1

In [ ]:
# 3000 images in training (1500-1500)
# 1400 images in testing (700-700)

x=1
for f in sl:
  if x <= 1500:
    shutil.copy('/content/drive/My Drive/DM Course Project/Plot images/seizure/'+f,'/content/drive/My Drive/DM Course Project/temp data(plot images)/train/seizure')
  elif x<=2200:
    shutil.copy('/content/drive/My Drive/DM Course Project/Plot images/seizure/'+f,'/content/drive/My Drive/DM Course Project/temp data(plot images)/test/seizure')
  
  x+=1

x=1
for f in nsl:
  if x <= 1500:
    shutil.copy('/content/drive/My Drive/DM Course Project/Plot images/non-seizure/'+f,'/content/drive/My Drive/DM Course Project/temp data(plot images)/train/non-seizure')
  elif x<=2200:
    shutil.copy('/content/drive/My Drive/DM Course Project/Plot images/non-seizure/'+f,'/content/drive/My Drive/DM Course Project/temp data(plot images)/test/non-seizure')
  
  x+=1s

In [ ]:
len(os.listdir('/content/drive/My Drive/DM Course Project/temp data(plot images)/train/seizure'))

## Essential Imports

In [ ]:
import tensorflow as tf
from keras.regularizers import l2
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import load_model
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

## Essential Constants

In [ ]:
train_path = '/content/drive/My Drive/DM Course Project/Temp data(plot images)/temp data(plot images)/train/'
test_path = '/content/drive/My Drive/DM Course Project/Temp data(plot images)/temp data(plot images)/test/'

# useful for getting number of classes
folders = glob('/content/drive/My Drive/DM Course Project/Temp data(plot images)/temp data(plot images)/train/*')

num_classes=len(folders)

# re-size all the images to this
IMAGE_SIZE = [299, 299]

In [ ]:
num_classes

2

## Creating a new model

In [ ]:
model = Sequential()
model.add(InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(1, kernel_regularizer=l2(0.01),activation='linear')) # Final Layer is SVM Layer
model.layers[0].trainable = False

87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
model.compile(optimizer='adam', loss='hinge', metrics=['accuracy',tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.MeanSquaredError(),tf.keras.metrics.Hinge(),tf.keras.metrics.BinaryCrossentropy()])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 8, 8, 2048)        21802784  
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               16777344  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 38,580,257
Trainable params: 16,777,473
Non-trainable params: 21,802,784
_________________________________________________________________


## Setting up training and testing data

In [ ]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
train_set = data_generator.flow_from_directory(train_path,target_size = (299, 299),batch_size=10,class_mode = 'binary')
test_set = data_generator.flow_from_directory(test_path,target_size = (299, 299),batch_size=10,class_mode = 'binary')

Found 3000 images belonging to 2 classes.
Found 1400 images belonging to 2 classes.


In [ ]:
print(test_set)

In [ ]:
r = model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(train_set),
  validation_steps=len(test_set)
)

f=open('/content/drive/My Drive/DM Course Project/Transfer Learning Model/TL-PI-Inceptionv3-stats.txt','w')
f.write(str(r.history))
f.close()

model.save('/content/drive/My Drive/DM Course Project/Transfer Learning Model/TL-PI-Inceptionv3-model.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
300/300 [==============================] - 1666s 5s/step - loss: 5.5849 - accuracy: 0.7249 - mean_absolute_error: 30.1961 - mean_squared_error: 1495.3306 - hinge: 5.5698 - binary_crossentropy: 4.1496 - val_loss: 3.7351 - val_accuracy: 0.8200 - val_mean_absolute_error: 41.7627 - val_mean_squared_error: 2841.7480 - val_hinge: 3.7242 - val_binary_crossentropy: 2.7359
Epoch 2/20
300/300 [==============================] - 58s 193ms/step - loss: 1.9200 - accuracy: 0.8707 - mean_absolute_error: 40.5814 - mean_squared_error: 2492.4213 - hinge: 1.9098 - binary_crossentropy: 1.9241 - val_loss: 1.6926 - val_accuracy: 0.8929 - val_mean_absolute_error: 33.7896 - val_mean_squared_error: 1801.9965 - val_hinge: 1.6839 - val_binary_crossentropy: 1.5874
Epoch 3/20
300/300 [==============================] - 57s 191ms/step - loss: 0.5061 - accuracy: 0.9471 - mean_absolute_error: 30.9900 - mean_squared_error: 1322.4344 - hinge: 0.4977 - binary_crossentropy: 0.7741 - val_loss: 2.1804 - val_accura

In [ ]:
import os
import cv2
bad_list=[]
dir='/content/drive/My Drive/DM Course Project/Temp data(plot images)/temp data(plot images)/'

subdir_list=os.listdir(dir) # create a list of the sub directories in the directory ie train or test
for d in subdir_list:  # iterate through the sub directories train and test
    dpath=os.path.join (dir, d) # create path to sub directory
    if d in ['train', 'test']:
        class_list=os.listdir(dpath) # list of classes ie dog or cat
       # print (class_list)
        for klass in class_list: # iterate through the two classes
            class_path=os.path.join(dpath, klass) # path to class directory
            #print(class_path)
            file_list=os.listdir(class_path) # create list of files in class directory
            for f in file_list: # iterate through the files
                fpath=os.path.join (class_path,f)
                index=f.rfind('.') # find index of period infilename
                ext=f[index+1:] # get the files extension
                if ext  not in ['jpg', 'png', 'bmp', 'gif']:
                    print(f'file {fpath}  has an invalid extension {ext}')
                    bad_list.append(fpath)                    
                else:
                    try:
                        img=cv2.imread(fpath)
                        size=img.shape
                    except:
                        print(f'file {fpath} is not a valid image file ')
                        bad_list.append(fpath)
                       
print (bad_list)

[]
